In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/stream.db',True)
dbutils.fs.rm('dbfs:/FileStore/streaming',True)

Out[1]: True

In [0]:
%sql
DROP DATABASE IF EXISTS stream CASCADE;
CREATE DATABASE IF NOT EXISTS stream

In [0]:
from pyspark.sql.types import StructType, StructField, StringType , IntegerType, FloatType

schema = StructType([   
                     StructField('Country',StringType()),
                     StructField('Citizens',IntegerType())
])

source_dir = 'dbfs:/FileStore/streaming/'

#### Reading the streaming dataframe

In [0]:
df = spark.readStream.format("csv")\
        .option('header','true')\
        .schema(schema)\
        .load(source_dir)


#### 01. Trigger - default or unspecifed Trigger

In [0]:
 WriteStream = ( df.writeStream
        .option('checkpointLocation',f'{source_dir}/AppendCheckpoint')
        .outputMode("append")
        .queryName('DefaultTrigger')
        .toTable("stream.AppendTable"))


#### 02. Trigger - processingTime

In [0]:
 WriteStream = ( df.writeStream
        .option('checkpointLocation',f'{source_dir}/AppendCheckpoint')
        .outputMode("append")
        .trigger(processingTime='2 minutes')
        .queryName('ProcessingTime')
        .toTable("stream.AppendTable"))


#### 03. Trigger - availablenow

In [0]:
 WriteStream = ( df.writeStream
        .option('checkpointLocation',f'{source_dir}/AppendCheckpoint')
        .outputMode("append")
        .trigger(availableNow=True)
        .queryName('AvailableNow')
        .toTable("stream.AppendTable"))

In [0]:
%sql
SELECT * FROM stream.AppendTable

Country,Citizens
India,10
USA,10
China,20
Brazil,10


In [0]:
%sql
drop table stream.AppendTable